<a href="https://colab.research.google.com/github/hinaabbaskhan/fashion-mnist-clothing-classifier-neural-network/blob/main/improve_mnist_with_convilutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Improve MNIST with Convolutions


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

## Load the data

Begin by loading the data.

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)`

In [3]:
# Load the data

# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz")

# Get only training set
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)

## Pre-processing the data

One important step when dealing with image data is to preprocess the data. During the preprocess step we apply transformations to the dataset that will be fed into our convolutional neural network.

Here we will apply two transformations to the data:
- Reshape the data so that it has an extra dimension. The reason for this is that commonly we use 3-dimensional arrays (without counting the batch dimension) to represent image data. The third dimension represents the color using RGB values. This data might be in black and white format so the third dimension doesn't really add any additional information for the classification process but it is a good practice regardless.


- Normalize the pixel values so that these are values between 0 and 1. We can achieve this by dividing every value in the array by the maximum.

These tensors are of type `numpy.ndarray` so we can use functions like [reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html) or [divide](https://numpy.org/doc/stable/reference/generated/numpy.divide.html) to complete the `reshape_and_normalize` function below:

In [4]:
def reshape_and_normalize(images):

    # Reshape the images to add an extra dimension
    images = np.reshape(images, (images.shape[0], images.shape[1], images.shape[2], 1))

    # Normalize pixel values
    images = np.divide(images,255)

    return images

In [5]:
# Reload the images in case you run this cell multiple times
(training_images, _), _ = tf.keras.datasets.mnist.load_data(path=data_path)

# Apply your function
training_images = reshape_and_normalize(training_images)

print(f"Maximum pixel value after normalization: {np.max(training_images)}\n")
print(f"Shape of training set after reshaping: {training_images.shape}\n")
print(f"Shape of one image after reshaping: {training_images[0].shape}")


Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)


**Expected Output:**
```
Maximum pixel value after normalization: 1.0

Shape of training set after reshaping: (60000, 28, 28, 1)

Shape of one image after reshaping: (28, 28, 1)
```

## Defining our callback

The callback will ensure that training will stop after an accuracy of 99.5% is reached:

In [6]:
class myCallback(tf.keras.callbacks.Callback):

  def on_epoch_end(self, epoch, logs={}):

    # Check the loss
    if(logs.get('accuracy') > 0.995):

      # Stop if threshold is met
      print("\nAccuracy is greater than 0.995 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()


## Convolutional Model


In [7]:
def convolutional_model():

    # Define the model
    model = tf.keras.models.Sequential([
     # Add convolutions and max pooling
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(2, 2),


      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])


    # Compile the model
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [8]:
# Save your untrained model
model = convolutional_model()

# Instantiate the callback class
callbacks = myCallback()

# Train your model (this can take up to 5 minutes)
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])

Epoch 1/10
1875/1875 [==============================] - 39s 20ms/step - loss: 0.1498 - accuracy: 0.9549
Epoch 2/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0501 - accuracy: 0.9851
Epoch 3/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0322 - accuracy: 0.9902
Epoch 4/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0200 - accuracy: 0.9937
Epoch 5/10
1875/1875 [==============================] - ETA: 0s - loss: 0.0132 - accuracy: 0.9956
Accuracy is greater than 0.995 so cancelling training!
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0132 - accuracy: 0.9956


If we see the message that we defined in your callback printed out after less than 10 epochs it means our callback worked as expected. We can also double check by running the following cell:



In [9]:
print(f"Our model was trained for {len(history.epoch)} epochs")

Our model was trained for 5 epochs
